In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
model=VGG16(weights="imagenet",include_top=False,input_shape=(224,224,3))
model.summary()

In [ ]:
model.summary()

In [ ]:
model.layers[0].__class__.__name__

In [ ]:
for l in model.layers:
    l.trainable=False

In [ ]:
for (i,l) in enumerate(model.layers):
    print(str(i) +" "+ l.__class__.__name__, l.trainable)

In [ ]:
for l in model.layers:
    l.trainable=False
def addTop(bottom_model, num_classes):
    """creates the top or head of the model that will be placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Softmax,Flatten,Activation,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

In [ ]:
print(model.summary())

In [ ]:
num_classes=2
FC_Head=addTop(model,num_classes)

In [ ]:
model_new=Model(inputs=model.input,output=FC_Head)
print(model_new.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_dir =("C:/Users/NARESH/Desktop/mlops-ws/FaceRecog/train/")
validation_data_dir =("C:/Users/NARESH/Desktop/mlops-ws/FaceRecog/validation/")

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_batch_size = 16
test_batch_size=10

In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(224, 224),
                                                    batch_size=train_batch_size,
                                                    class_mode='categorical',
                                                   shuffle=False)

In [ ]:
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=(224, 224),
                                                              batch_size=test_batch_size,
                                                              class_mode='categorical')

In [ ]:
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint("facerecognition_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

In [ ]:
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

In [ ]:
# we put our call backs into a callback list
callbacks =[earlystop, checkpoint]

In [ ]:
# We use a very small learning rate
model_new.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr=0.001),
              metrics = ['accuracy'])

In [ ]:
nb_train_samples =1990
nb_validation_samples = 100
epochs = 4
batch_size = 10

In [ ]:
history = model_new.fit_generator(train_generator,
                              steps_per_epoch = nb_train_samples // batch_size,
                              epochs = epochs,
                              callbacks = callbacks,
                              validation_data = validation_generator,
                              validation_steps = nb_validation_samples // batch_size   )
model_new.save("facerecognition_vgg.h5")
    

In [ ]:
import cv2
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from PIL import Image
   
face_rec_model = load_model('facerecognition_vgg.h5')

In [ ]:
cap=cv2.VideoCapture(0)

face_model=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extract(img):
    face_cor=face_model.detectMultiScale(photo,1.3,5)
    if face_cor is ():
        return None
    for (x,y,z,h) in face_cor:
        cv2.rectangle(img,(x,y),(x+z,y+h),(0,255,255),2)
        cropped_face=img[y:y+h, x:x+z]
    return cropped_face

In [ ]:
cap=cv2.VideoCapture(0)

In [ ]:
while True:
    status,photo=cap.read()
    face=face_extract(photo)
    if type(face) is np.ndarray:
        face=cv2.resize(face,(224,224))
        im=Image.fromarray(face,'RGB')
        img_array=np.array(im)
        img_array=np.expand_dims(img_array,axis=0)
        pred=face_rec_model.predict(img_array)
        print(pred)
        name="Not matching"
        
        if(pred[0][0]>0.5):
            name='Kritika'
        cv2.putText(photo,name,(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    else:
        cv2.putText(photo,"No face found",(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow("Video",photo)
    if cv2.waitKey(10)==13:
            break
cv2.destroyAllWindows()